# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In [1044]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import glob
from itertools import chain
pd.set_option('display.max_rows', None)

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.

In [1045]:
#First, we know that all the file are excel. 
files = os.listdir(r'../../_data/sbs/B_RawData/bancos/')   #Save in a list all the names of all the files         
y = [z[12:-5] for z in files] 

In [1046]:
#This function is useful to rename duplicates columns. 
def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [1047]:
li = {}    #Create an empty dictionary 
for f in y:
    li["table_"+f] = pd.read_excel(r'../../_data/sbs/B_RawData/bancos/table_clean_'+f+'.xlsx') #We save every data in li
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_9_2002", "%d_%m_%Y").date():   #For the datas in these year need transponse
            li["table_"+f] = li["table_"+f].iloc[1:].T.reset_index().set_axis(li["table_"+f].iloc[1:].T.reset_index().iloc[0], axis=1).iloc[1:]
            li["table_"+f]["date_info"] = datetime.strptime(f, "%d_%m_%Y").date() #We add the column "date_info" in every dataframe
            df_column_uniquify(li["table_"+f])
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[0]:'TIPO', li["table_"+f].columns[1]:'TASA ANUAL (%)', 'Promedio del Sistema':'PROMEDIO'}) #Transponse
            
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_5_2008", "%d_%m_%Y").date(): 
            li["table_"+f]["PROMEDIO"]  = np.where(li["table_"+f][np.nan].isna() == False,
                                             li["table_"+f][np.nan], li["table_"+f]["PROMEDIO"] )  #We have to fix the column 'PROMEDIO'
            li["table_"+f].drop(columns = [np.nan], axis = 1, inplace = True)  #Drop the column nan where we  had values of PROMEDIO. 
            
    else: 
            li["table_"+f] =  li["table_"+f].rename(columns=str.upper)
            li["table_"+f]["date_info"] = datetime.strptime(f, "%d_%m_%Y").date()
            li["table_"+f] = li["table_"+f][~li["table_"+f]["TASA ANUAL (%)"].str.contains('|'.join(["Nota"]))]  #Drop row that has the word "Nota"

2. Append all this datasets and generate a unique dataframe. This new dataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**

In [1048]:
#Create a new dictonary only with the types of rates 
holi = {}
for f in y:
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_9_2002", "%d_%m_%Y").date():
        holi["tipo_"+f] = li["table_"+f][['TIPO','TASA ANUAL (%)','date_info']]
        li["table_"+f] = li["table_"+f].drop(li["table_"+f].columns[0], axis = 1)
        

In [1049]:
#APPEND THE DATA WITH THE TYPES 
df3 = pd.concat(holi.values(), axis=0).reset_index(drop=True).sort_values(by=['date_info']).drop_duplicates(subset=['TASA ANUAL (%)'])
len(df3)

32

In [1050]:
data = [(li["table_"+x].columns.values.tolist(), li["table_"+x].iloc[0,-1]) for x in y] # to use the second column and "date" of "li" and create a tuple of the two values to append it  
data.sort(key=lambda x: x[1])  # to order the data by the date

for col, date in data: # loop for generate col and date 
    print(f'Columns: [{col}], date: [{date}]') # print the result

Columns: [['TASA ANUAL (%)', 'BBVA', 'COMERCIO', 'CRÉDITO', 'PICHINCHA', 'BIF', 'SCOTIABANK', 'CITIBANK', 'INTERBANK', 'MIBANCO', 'GNB', 'FALABELLA', 'SANTANDER', 'RIPLEY', 'ALFIN', 'ICBC', 'BANK OF CHINA', 'PROMEDIO', 'date_info']], date: [2001-03-31]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO', 'BANCO PICHINCHA', 'BANBIF', 'B SANTANDER CENTRAL', 'STANDARD CHARTERED', 'BANCO SUDAMERICANO', 'WIESE SUDAMERIS', 'BANK BOSTON', 'BNP PARIBAS EL', 'CITIBANK DEL PERU', 'INTERBANK', 'MIBANCO', 'PROMEDIO', 'DATE_INFO', 'date_info']], date: [2002-09-30]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO', 'BANCO PICHINCHA', 'BANBIF', 'B SANTANDER CENTRAL', 'STANDARD CHARTERED', 'BANCO SUDAMERICANO', 'WIESE SUDAMERIS', 'BANK BOSTON', 'BNP PARIBAS EL', 'CITIBANK DEL PERU', 'INTERBANK', 'MIBANCO', 'PROMEDIO', 'DATE_INFO', 'date_info']], date: [2002-10-31]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCI

In [1070]:
#We made a loop to rename the names of banks 
for f in y:
    for x in range(len(li["table_"+f].axes[1])):
        if li["table_"+f].columns[x] in ('FINANCIERO', 'BANCO PICHINCHA'):  
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'PICHINCHA'})
        if li["table_"+f].columns[x] in ('BBVA', 'CONTINENTAL'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'BBVA'})
        if li["table_"+f].columns[x] in ('BANCO DE COMERCIO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'COMERCIO'})
        if li["table_"+f].columns[x] in ('B SANTANDER CENTRAL'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'SANTANDER'})
        if li["table_"+f].columns[x] in ('AZTECA'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'ALFIN'})
        if li["table_"+f].columns[x] in ('SCOTIABANK PERU', 'BANCO SUDAMERICANO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'SCOTIABANK'})
        if li["table_"+f].columns[x] in ('BANCO GNB', 'HSBC'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'GNB'})
        if li["table_"+f].columns[x] in ('BANCO DE CREDITO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'CRÉDITO'})
        if li["table_"+f].columns[x] in ('CITIBANK DEL PERU'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'CITIBANK'})
       

In [1071]:
#We check the changes 
li["table_30_12_2009"]

,TASA ANUAL (%),BBVA,BANCO DE COMERCIO,CRÉDITO,PICHINCHA,BANBIF,SCOTIABANK,CITIBANK,INTERBANK,MIBANCO,GNB,BANCO FALABELLA,SANTANDER PERU,BANCO RIPLEY,ALFIN BANCO,DEUTSCHE BANK PERU,PROMEDIO,date_info
1,Avances en Cuenta Corriente,16.05,-,41.05,-,25.57,6.47,4.30,5.30,-,-,-,-,-,-,-,23.56,2009-12-30
2,Sobregiros,48.68,138.18,78.72,-,82.42,51.97,10.01,12.57,-,60.00,-,-,-,-,-,64.60,2009-12-30
3,Descuentos,14.75,15.66,6.03,9.09,7.01,7.26,-,7.10,-,6.50,-,-,-,-,-,7.66,2009-12-30
4,Clientes deuda > S/. 5 MM4,3.82,5.50,3.12,12.17,5.53,4.12,4.64,5.29,-,6.38,-,-,-,-,-,3.63,2009-12-30
5,Demás Clientes,9.34,16.39,4.96,17.67,7.99,5.74,3.92,5.56,24.57,5.64,2.25,5.71,-,-,-,7.37,2009-12-30
6,Promedio,6.39,14.45,3.84,14.58,6.75,5.30,4.55,5.49,24.57,6.10,2.25,5.71,-,-,-,5.32,2009-12-30
7,Clientes deuda > S/. 5 MM,1.82,-,2.15,-,-,-,4.02,6.59,-,2.95,-,-,-,-,-,2.05,2009-12-30
8,Demás Clientes,9.65,13.10,5.14,13.00,2.85,2.96,-,8.30,-,-,-,2.39,-,-,-,5.91,2009-12-30
9,Promedio,3.00,13.10,2.83,13.00,2.85,2.96,4.02,7.21,-,2.95,-,2.39,-,-,-,3.04,2009-12-30
10,Clientes deuda > S/. 5 MM,3.47,-,3.07,8.87,5.93,3.04,4.49,3.88,-,3.53,-,-,-,-,-,3.41,2009-12-30


In [1075]:
#APPEND THE DATA 
DATA = pd.concat(li.values(), axis=0).reset_index(drop=True).sort_values(by=['date_info'])

3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.

4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**